In [68]:
import pandas as pd

# 读取数据
financial_ratio = pd.read_csv('/Users/shawn/Github/M1/金融機構與風險管理/S_P500 財務比率.csv')
rating = pd.read_csv('/Users/shawn/Github/M1/金融機構與風險管理/rating.csv')

financial_ratio['datadate'] = pd.to_datetime(financial_ratio['datadate'], format='%Y/%m/%d')
financial_ratio['datadate'] = financial_ratio['datadate'].dt.strftime('%Y-%m-%d')


rating['datadate'] = pd.to_datetime(rating['datadate'], format='%Y-%m-%d')
rating['datadate'] = rating['datadate'].dt.strftime('%Y-%m-%d')

In [69]:
merged_df = pd.merge(financial_ratio, rating[['datadate', 'ticker', 'splticrm']], on=['datadate', 'ticker'], how='left')
merged_df['splticrm']


print(merged_df[merged_df['ticker'] == 'AMD'][['datadate','ticker','splticrm']].head()) 
print(rating[rating['ticker'] == 'AMD'][['datadate','ticker','splticrm']].head()) 

         datadate ticker splticrm
31998  2010-01-31    AMD       B-
31999  2010-02-28    AMD       B-
32000  2010-03-31    AMD       B-
32001  2010-04-30    AMD       B-
32002  2010-05-31    AMD       B-
       datadate ticker splticrm
388  2001-01-31    AMD        B
389  2001-02-28    AMD        B
390  2001-03-31    AMD        B
391  2001-04-30    AMD        B
392  2001-05-31    AMD        B


In [73]:
# financial_ratio, rating期間範圍無對齊
merged_df.dropna(subset=['splticrm'], inplace=True)
def filter_rows(group):
    # 組內rating前後不同的才會被留下
    mask = group['splticrm'] != group['splticrm'].shift()
    return group[mask]
merged_df = merged_df.groupby('ticker').apply(filter_rows).reset_index(drop=True)
merged_df['ticker'].value_counts().to_frame() 


,count
ticker,
CHK,12
AKS,7
ETFC,7
M,7
DO,7
...,...
JNJ,1
KEY,1
KMB,1


In [77]:
# merged_df保留了所有ticker
merged_df2 = merged_df.groupby('ticker').filter(lambda x: len(x) > 1)
print(merged_df2['ticker'].value_counts().to_frame())
print(len(merged_df2))
merged_df2.to_csv('SP500_merge.csv')

        count
ticker       
CHK        12
ETFC        7
DO          7
AKS         7
M           7
...       ...
OKE         2
D           2
ODP         2
DD          2
PEG         2

[225 rows x 1 columns]
668
